# Multiple resonators on a chip, with possibility of having holes in the resonator core

Ruirements: gdspy library.
The design is generated in the following order:


   *   **Ground and contact pads**. THe idea is that you should only specify the coordinates of the contact pads; whether they should be rotated should be decided by the code itself (if the contact pad is further that 900 $\mu m$ away, it's rotated). This is done by the functions 
   
   *Generate_Pads (sample_size_along_x_axis, sample_size_along_y_axis, [d], [d1], [d2], [layer], [contact pad coordinates])*. 
   
   
   *   **TL**. Connects two contact pads from left to right. Generally, in this code everything that involves coplanar waveguides should be drawn from left to right.
   
   *Generate_TL (contact_pad_1, contact_pad2, d, d1, d2)*
   
    
   *   **Resonators** The position is specified by the left lower corner in (x,y) format. 
   
       *Generate_Resonators (initial_point_array, [circular_claw_radius], [gap], [resonator_frequency], [mode (up or down)], [layer])*.
   
   *   **DC line**. At this point the we add resonators and dc bias lines to the sample cell.
   
   *Generate_Loops* ([coordinates of the second end; the first end automatically gets connected to a contact pad], [modes], [d], [d1], [d2], [d for the contact pad], [d1 for the contact pad], [d2 for the contact pad], [array of arc coordinates], [array of arc angles], layer)
   
   
   *   **Ground grid**. Can be generated only after everything else is in place
   
   *Generate_Grid* (sample_size_along_x, sample_size_along_y, layer1, layer2)
   
 IF you want to change anything, unfortunately you'll have to do a restart.

In [1]:
!pip install gdspy

In [2]:
from design_res3 import *
from resonators3_2 import*
import gdspy
import random as rd

In [3]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(TL.GenerateTL(d, coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL(d1, coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL(d2, coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs):
        self.RESONATORS1 = gdspy.Cell('RESONATORS1')
        self.RESONATORS2 = gdspy.Cell('RESONATORS2')
        self.RESONATORS3 = gdspy.Cell('RESONATORS3')
        self.RESONATORS_RESTRICTED = gdspy.Cell('RESONATORS_RESTRICTED')
        #self.resonators = []
        #self.res_cells = []
        self.res_references = []
        for i, (start, freq, mode, w, s, TLi, TLv, TLg, DE) in enumerate(zip(start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs)):
            #self.resonators.append(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE))
            #self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.RESONATORS1.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate1(mode))
            self.RESONATORS2.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate2(mode))
            self.RESONATORS3.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            self.RESONATORS_RESTRICTED.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            #self.res_cells[i].add(self.resonators[i].Generate3(mode))
            
        COMBINATION = gdspy.Cell('COMBINATION')
        COMBINATION.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS3, (0,0)), gdspy.CellReference(self.tl_cell3, (0,0)), 'or'))
        CUT = gdspy.Cell('CUT')
        CUT.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS2, (0,0)), gdspy.CellReference(self.tl_cell2, (0,0)), 'or'))
        CUTOUT = gdspy.Cell('CUTOUT')
        CUTOUT.add(gdspy.boolean(gdspy.CellReference(COMBINATION, (0,0)), gdspy.CellReference(CUT, (0,0)), 'not'))
        FILL = gdspy.Cell('FILL')
        FILL.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS1, (0,0)), gdspy.CellReference(self.tl_cell1, (0,0)), 'or'))
        
        CUTOUTFILL = gdspy.Cell('CUTOUTFILL')
        CUTOUTFILL.add(gdspy.boolean(gdspy.CellReference(CUTOUT, (0,0)), gdspy.CellReference(FILL, (0,0)), 'or'))
        self.cell.add(gdspy.CellReference(CUTOUTFILL, (0, 0)))

        
                        
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (405, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 5, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 405), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 5), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        area = gdspy.CellReference(self.RESONATORS_RESTRICTED, (0,0))
        res1 = gdspy.boolean(res1, area, 'not', layer = l1)
        res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    #self.Resonators
    #self.JJ
    #self.Loops
    #self.TL

In [4]:
TL_inner = 12
TL_vacuum = 2
TL_ground = 22
TL_total = TL_inner + 2*(TL_vacuum + TL_ground)
number = 72
w = 42.5
s = 11
DE = 25.5
positions = [] 
for j in range(3):
    for i in range(12):
        if j == 0:
            positions.append(coordinates(800 + 50 + 710*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(800 + 50 + 710*i, 3900 + TL_total/2 - j*1400))
        elif j == 1:
            positions.append(coordinates(680 + 710*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(680 + 710*i, 3900 + TL_total/2 - j*1400))
        elif j == 2:
            positions.append(coordinates(590 + 710*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(590 + 710*i, 3900 + TL_total/2 - j*1400)) 
w_arr = np.full(number, w) 
s_arr = np.full(number, s) 
f_arr = np.linspace(8e9, 9e9, number)
rd.shuffle(f_arr)
print(f_arr)
DE_arr = np.full(number, DE)
TL_inner_arr = np.repeat(TL_inner, number)
TL_vacuum_arr = np.repeat(TL_vacuum, number)
TL_ground_arr = np.repeat(TL_ground, number)
ground_arr = np.repeat('ground', number)
updown = ['up', 'down']
updown_arr = np.tile(updown, number)

[8.98591549e+09 8.73239437e+09 8.78873239e+09 8.97183099e+09
 8.02816901e+09 8.76056338e+09 8.87323944e+09 8.60563380e+09
 8.59154930e+09 8.25352113e+09 8.54929577e+09 8.90140845e+09
 8.05633803e+09 8.33802817e+09 8.42253521e+09 8.61971831e+09
 8.35211268e+09 8.56338028e+09 8.21126761e+09 8.81690141e+09
 8.39436620e+09 8.83098592e+09 8.00000000e+09 8.85915493e+09
 8.71830986e+09 8.07042254e+09 8.50704225e+09 8.95774648e+09
 8.12676056e+09 8.74647887e+09 8.09859155e+09 8.46478873e+09
 8.70422535e+09 8.14084507e+09 8.57746479e+09 8.67605634e+09
 8.26760563e+09 8.08450704e+09 8.45070423e+09 8.80281690e+09
 8.38028169e+09 8.69014085e+09 8.64788732e+09 8.53521127e+09
 8.43661972e+09 8.15492958e+09 8.30985915e+09 8.63380282e+09
 8.49295775e+09 8.32394366e+09 8.40845070e+09 8.04225352e+09
 8.16901408e+09 8.91549296e+09 8.28169014e+09 8.92957746e+09
 8.22535211e+09 8.94366197e+09 8.18309859e+09 8.52112676e+09
 8.47887324e+09 8.36619718e+09 8.11267606e+09 8.19718310e+09
 8.01408451e+09 8.887323

In [5]:
sample = Sample(2)

sample.Generate_Pads(10000, 5000, #width, depth
                     2, #number of Pads
                     [TL_inner, TL_inner], #d
                     [TL_inner +2*TL_vacuum, TL_inner +2*TL_vacuum], #d1
                     [TL_total, TL_total], #d2
                     0, #layer
                     [coordinates(800, 3900-TL_total/2), coordinates(9200, 1100-TL_total/2)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, TL_inner, TL_inner +2*TL_vacuum, TL_inner + 2*(TL_vacuum+TL_ground), 0)

sample.Generate_Resonators(positions, # ref coordinates
                           f_arr, #freqs
                           updown_arr, #mode
                           w_arr, #w
                           s_arr, #s
                           TL_inner_arr,
                           TL_vacuum_arr,
                           TL_ground_arr,
                           DE_arr) 


coupling length: 478.74594503287705
x coordinate of end of resonator 1504.2209450328771
DE= 25.5
...
coupling length: 478.74594503287705
x coordinate of end of resonator 1504.2209450328771
DE= 25.5
...
coupling length: 478.74594503287705
x coordinate of end of resonator 1504.2209450328771
DE= 25.5
...
coupling length: 478.74594503287705
x coordinate of end of resonator 1504.2209450328771
DE= 25.5
...
coupling length: 492.64502085641215
x coordinate of end of resonator 1518.1200208564123
DE= 25.5
...
coupling length: 492.64502085641215
x coordinate of end of resonator 1518.1200208564123
DE= 25.5
...
coupling length: 492.64502085641215
x coordinate of end of resonator 1518.1200208564123
DE= 25.5
...
coupling length: 492.64502085641215
x coordinate of end of resonator 1518.1200208564123
DE= 25.5
...
coupling length: 489.4870399534865
x coordinate of end of resonator 2224.9620399534865
DE= 25.5
...
coupling length: 489.4870399534865
x coordinate of end of resonator 2224.9620399534865
DE= 2

coupling length: 487.14499670012054
x coordinate of end of resonator 8612.61999670012
DE= 25.5
...
coupling length: 487.14499670012054
x coordinate of end of resonator 8612.61999670012
DE= 25.5
...
coupling length: 537.7463255827034
x coordinate of end of resonator 9373.221325582703
DE= 25.5
...
coupling length: 537.7463255827034
x coordinate of end of resonator 9373.221325582703
DE= 25.5
...
coupling length: 537.7463255827034
x coordinate of end of resonator 9373.221325582703
DE= 25.5
...
coupling length: 537.7463255827034
x coordinate of end of resonator 9373.221325582703
DE= 25.5
...
coupling length: 485.5960459951916
x coordinate of end of resonator 9321.071045995192
DE= 25.5
...
coupling length: 485.5960459951916
x coordinate of end of resonator 9321.071045995192
DE= 25.5
...
coupling length: 485.5960459951916
x coordinate of end of resonator 9321.071045995192
DE= 25.5
...
coupling length: 485.5960459951916
x coordinate of end of resonator 9321.071045995192
DE= 25.5
...
coupling l

coupling length: 504.02625896200595
x coordinate of end of resonator 7749.5012589620055
DE= 25.5
...
coupling length: 504.02625896200595
x coordinate of end of resonator 7749.5012589620055
DE= 25.5
...
coupling length: 509.916382188607
x coordinate of end of resonator 8465.391382188607
DE= 25.5
...
coupling length: 509.916382188607
x coordinate of end of resonator 8465.391382188607
DE= 25.5
...
coupling length: 509.916382188607
x coordinate of end of resonator 8465.391382188607
DE= 25.5
...
coupling length: 509.916382188607
x coordinate of end of resonator 8465.391382188607
DE= 25.5
...
coupling length: 527.5300741467627
x coordinate of end of resonator 8483.005074146762
DE= 25.5
...
coupling length: 527.5300741467627
x coordinate of end of resonator 8483.005074146762
DE= 25.5
...
coupling length: 527.5300741467627
x coordinate of end of resonator 8483.005074146762
DE= 25.5
...
coupling length: 527.5300741467627
x coordinate of end of resonator 8483.005074146762
DE= 25.5
...
coupling l

coupling length: 484.0569143121641
x coordinate of end of resonator 6929.531914312163
DE= 25.5
...
coupling length: 484.0569143121641
x coordinate of end of resonator 6929.531914312163
DE= 25.5
...
coupling length: 486.3692881066491
x coordinate of end of resonator 7641.8442881066485
DE= 25.5
...
coupling length: 486.3692881066491
x coordinate of end of resonator 7641.8442881066485
DE= 25.5
...
coupling length: 486.3692881066491
x coordinate of end of resonator 7641.8442881066485
DE= 25.5
...
coupling length: 486.3692881066491
x coordinate of end of resonator 7641.8442881066485
DE= 25.5
...
coupling length: 496.65026492841554
x coordinate of end of resonator 7652.125264928415
DE= 25.5
...
coupling length: 496.65026492841554
x coordinate of end of resonator 7652.125264928415
DE= 25.5
...
coupling length: 496.65026492841554
x coordinate of end of resonator 7652.125264928415
DE= 25.5
...
coupling length: 496.65026492841554
x coordinate of end of resonator 7652.125264928415
DE= 25.5
...
co

In [6]:
#gdspy.LayoutViewer(depth = 2)

In [7]:
sample.Generate_Grid(10000,5000,0,0)

In [8]:
gdspy.write_gds('resonators3_2_new2.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)